In [14]:
%pip uninstall numpy -y
%pip install numpy==1.26.4

Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install pyspark==3.5.1
%pip install pandas
%pip install findspark
%pip install joblib
%pip install scikit-learn
%pip install pyvi
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 40.2 MB/s  0:00:00m0:00:010:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
# PATH definitions
DATA_DIR = "./dataset"  # đổi nếu dùng Drive
MODEL_DIR = "./torch_models"
TRAIN_PATH = os.path.join(DATA_DIR, "train_data.csv")
VAL_PATH   = os.path.join(DATA_DIR, "val_data.csv")
TEST_PATH  = os.path.join(DATA_DIR, "test_data.csv")

In [3]:
# Columns
TEXT_COL = "Review"
ASPECTS  = ["Price","Shipping","Outlook","Quality","Size","Shop_Service","General","Others"]

# Label mapping
SENT_ID2NAME = {-1: "None", 0: "Negative", 1: "Positive", 2: "Neutral"}
LABEL_VALUES = list(SENT_ID2NAME.keys())  # [-1, 0, 1, 2]
LABEL_NAMES  = list(SENT_ID2NAME.values())  # ["None","Negative","Positive","Neutral"]

SEQ_LEN = 50        # max number of words per review
EMBEDDING_DIM = 300 # embedding vector size
MAX_WORDS = 20000

In [5]:
# Utilities function 
import re, unicodedata
import pandas as pd
from pyvi import ViTokenizer

URL_RE = re.compile(r"https?://\S+|www\.\S+")
TAG_RE = re.compile(r"<[^>]+>")
MULTISPACE_RE = re.compile(r"\s+")
VIETNAMESE_BASIC_STOPWORDS = set("""
và hoặc nhưng là thì mà được bị của cho với về từ tới đến nỗi do vì nên nếu khi để bằng như lại đã đang sẽ không chưa chẳng rất quá lắm hơi
này kia nọ đó đây ấy vậy thế sao tại vì do đó tuy nhiên hơn kém chỉ mỗi một các những cái con chiếc đôi đc nhé nha ạ ơi
""".split())

def preprocess_text(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip().lower()
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)
    text = re.sub(r"<[^>]+>", " ", text)
    text = text.replace("❤️", " yeu ").replace("❤", " yeu ").replace("😍", " yeu ")
    text = re.sub(r"[^\w\sáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ]", " ", text)
    text = unicodedata.normalize("NFC", text)
    text = MULTISPACE_RE.sub(" ", text)
    tokens = [w for w in ViTokenizer.tokenize(text).split() if w not in VIETNAMESE_BASIC_STOPWORDS]
    return " ".join(tokens)

In [20]:
def load_and_merge(train_path, val_path, test_path):
    train_df = pd.read_csv(train_path)
    val_df   = pd.read_csv(val_path)
    test_df  = pd.read_csv(test_path)

    # Merge train + val
    full_train = pd.concat([train_df, val_df], axis=0).reset_index(drop=True)

    # Preprocess review text
    full_train[TEXT_COL] = full_train[TEXT_COL].map(preprocess_text)
    test_df[TEXT_COL]    = test_df[TEXT_COL].map(preprocess_text)

    return full_train, test_df

In [4]:
%pip install torch torchtext --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 32.4 MB/s  0:00:18:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.0 MB/s  0:00:00
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cublas_c

In [21]:
import torch
from torchtext.vocab import build_vocab_from_iterator

SEQ_LEN = 50
MAX_WORDS = 20000

def yield_tokens(texts):
    for text in texts:
        yield text.split()

def build_vocab(texts):
    vocab = build_vocab_from_iterator(yield_tokens(texts), specials=["<unk>"])
    vocab.set_default_index(vocab["<unk>"])
    return vocab

def texts_to_sequences(texts, vocab, seq_len=SEQ_LEN):
    sequences = []
    for text in texts:
        tokens = text.split()
        seq = [vocab[token] for token in tokens]
        # pad or truncate
        if len(seq) < seq_len:
            seq = seq + [0]*(seq_len - len(seq))
        else:
            seq = seq[:seq_len]
        sequences.append(seq)
    return torch.tensor(sequences, dtype=torch.long)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class CNNTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, seq_len, num_classes, embedding_weights=None):
        super().__init__()
        if embedding_weights is not None:
            self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=False)
        else:
            self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=3)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(128, 64)
        self.out = nn.Linear(64, num_classes)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.embedding(x)           # [B, seq_len, embed_dim]
        x = x.permute(0, 2, 1)         # [B, embed_dim, seq_len]
        x = F.relu(self.conv(x))       # [B, 128, L]
        x = self.pool(x).squeeze(-1)   # [B, 128]
        x = F.relu(self.fc(x))         # [B, 64]
        x = self.dropout(x)
        return self.out(x)             # [B, num_classes]

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

def train_cnn_models(train_df, test_df, vocab, seq_len=50, embed_dim=300, batch_size=32, epochs=5):
    """
    Train CNN models for multi-aspect classification like XGB function structure.
    Returns:
        models: dict of trained CNN models per aspect
        encoders: dict of LabelEncoders per aspect
        reports: dict of sklearn classification reports per aspect
    """
    models = {}
    encoders = {}
    reports = {}

    # Convert train/test text to sequences once
    X_train_seq = texts_to_sequences(train_df[TEXT_COL], vocab, seq_len)
    X_test_seq  = texts_to_sequences(test_df[TEXT_COL], vocab, seq_len)

    for aspect in ASPECTS:
        print(f"\n=== Training aspect: {aspect} ===")

        # Encode labels
        le = LabelEncoder()
        y_train = torch.tensor(le.fit_transform(train_df[aspect]), dtype=torch.long)
        y_test  = torch.tensor(le.transform(test_df[aspect]), dtype=torch.long)
        encoders[aspect] = le

        # Build dataset & loader
        train_dataset = TensorDataset(X_train_seq, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        # Build CNN model
        model = CNNTextClassifier(vocab_size=len(vocab), embed_dim=embed_dim, seq_len=seq_len, num_classes=len(le.classes_))
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss()

        # Train
        for epoch in range(epochs):
            model.train()
            total_loss = 0
            for xb, yb in train_loader:
                optimizer.zero_grad()
                logits = model(xb)
                loss = criterion(logits, yb)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(train_loader):.4f}")

        # Evaluate
        model.eval()
        with torch.no_grad():
            logits_test = model(X_test_seq)
            preds_test = torch.argmax(logits_test, dim=1).numpy()

        report = classification_report(
            le.inverse_transform(y_test),
            le.inverse_transform(preds_test),
            labels=LABEL_VALUES,
            target_names=[SENT_ID2NAME[i] for i in LABEL_VALUES]
        )
        print(report)

        models[aspect] = model
        reports[aspect] = report

    return models, encoders, reports

In [23]:
def predict_aspects_cnn(models, encoders, texts, vocab, seq_len=50):
    """
    Predict multiple aspects for a list of texts using trained CNN models.
    Returns a pandas DataFrame with each aspect's prediction.
    """
    # Preprocess texts and convert to sequences
    sequences = texts_to_sequences([preprocess_text(t) for t in texts], vocab, seq_len)
    
    results = {}
    for aspect, model in models.items():
        model.eval()
        with torch.no_grad():
            logits = model(sequences)
            pred_encoded = torch.argmax(logits, dim=1).numpy()
        # Map back to original label
        pred_original = encoders[aspect].inverse_transform(pred_encoded)
        # Map to SENT_ID2NAME for display
        results[aspect] = [SENT_ID2NAME[int(x)] for x in pred_original]
    
    return pd.DataFrame(results, index=range(len(texts)))

In [24]:
# Example usage
train_df, test_df = load_and_merge(TRAIN_PATH, VAL_PATH, TEST_PATH)
vocab = build_vocab(train_df[TEXT_COL])
X_train_seq = texts_to_sequences(train_df[TEXT_COL], vocab)
X_test_seq  = texts_to_sequences(test_df[TEXT_COL], vocab)

In [ ]:

cnn_models, cnn_encoders, cnn_reports = train_cnn_models(
    train_df,
    test_df,
    vocab=vocab,
    seq_len=SEQ_LEN,
    embed_dim=EMBEDDING_DIM,
    batch_size=32,
    epochs=10
)


=== Training aspect: Price ===
Epoch 1/10 - Loss: 0.2821
Epoch 2/10 - Loss: 0.1679
Epoch 3/10 - Loss: 0.1234
Epoch 4/10 - Loss: 0.0819
Epoch 5/10 - Loss: 0.0505
Epoch 6/10 - Loss: 0.0355
Epoch 7/10 - Loss: 0.0309
Epoch 8/10 - Loss: 0.0211
Epoch 9/10 - Loss: 0.0228
Epoch 10/10 - Loss: 0.0147


/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

              precision    recall  f1-score   support

        None       0.96      0.96      0.96      1999
    Negative       0.00      0.00      0.00         3
    Positive       0.76      0.83      0.79       247
     Neutral       0.47      0.37      0.42        91

    accuracy                           0.92      2340
   macro avg       0.55      0.54      0.54      2340
weighted avg       0.92      0.92      0.92      2340


=== Training aspect: Shipping ===
Epoch 1/10 - Loss: 0.2974
Epoch 2/10 - Loss: 0.1627
Epoch 3/10 - Loss: 0.1151
Epoch 4/10 - Loss: 0.0761
Epoch 5/10 - Loss: 0.0499
Epoch 6/10 - Loss: 0.0345
Epoch 7/10 - Loss: 0.0247
Epoch 8/10 - Loss: 0.0252
Epoch 9/10 - Loss: 0.0183
Epoch 10/10 - Loss: 0.0202
              precision    recall  f1-score   support

        None       0.96      0.98      0.97      1635
    Negative       0.85      0.75      0.80       124
    Positive       0.92      0.94      0.93       549
     Neutral       0.12      0.03      0.05        3

/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

              precision    recall  f1-score   support

        None       0.90      0.89      0.90      1861
    Negative       0.00      0.00      0.00        11
    Positive       0.51      0.55      0.53       285
     Neutral       0.46      0.46      0.46       183

    accuracy                           0.81      2340
   macro avg       0.47      0.48      0.47      2340
weighted avg       0.81      0.81      0.81      2340


=== Training aspect: Others ===
Epoch 1/10 - Loss: 0.1610
Epoch 2/10 - Loss: 0.0762
Epoch 3/10 - Loss: 0.0326
Epoch 4/10 - Loss: 0.0193
Epoch 5/10 - Loss: 0.0133
Epoch 6/10 - Loss: 0.0146
Epoch 7/10 - Loss: 0.0124
Epoch 8/10 - Loss: 0.0053
Epoch 9/10 - Loss: 0.0071
Epoch 10/10 - Loss: 0.0041
              precision    recall  f1-score   support

        None       0.97      0.98      0.98      2151
    Negative       0.00      0.00      0.00         0
    Positive       0.00      0.00      0.00         0
     Neutral       0.78      0.65      0.71       189


/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/kaos/AfterGradEx/big_data/ABSA/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(aver

In [16]:

sample_texts = [
    "Giày rất đẹp, giá hợp lý nhưng giao hàng chậm",
    "Giá cao, giày xấu, shop giao hàng chậm"
]


print("=== CNN Predictions ===")
print(predict_aspects_cnn(cnn_models, cnn_encoders, sample_texts, vocab, seq_len=SEQ_LEN))

=== CNN Predictions ===
     Price  Shipping   Outlook Quality  Size Shop_Service General Others
0  Neutral  Negative  Positive    None  None         None    None   None
1     None  Negative  Negative    None  None         None    None   None


In [18]:
import joblib

# Save models
for aspect, model in cnn_models.items():
    model_path = os.path.join(MODEL_DIR, f"{aspect}_cnn_model.pt")
    torch.save(model.state_dict(), model_path)

# Save encoders (joblib is fine for these)
import joblib
for aspect, encoder in cnn_encoders.items():
    joblib.dump(encoder, os.path.join(MODEL_DIR, f"{aspect}_cnn_encoder.pkl"))

print("✅ All existing models and encoders saved to disk")

✅ All existing models and encoders saved to disk


In [34]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
# ----------------------
# UDF factory for each aspect
# ----------------------

def make_predict_udf(aspect, seq_len=50):
    model_path = os.path.join(MODEL_DIR, f"{aspect}_cnn_model.pt")
    encoder_path = os.path.join(MODEL_DIR, f"{aspect}_cnn_encoder.pkl")

     # Lazy load — only once per executor
    model = None
    encoder = None
    vocab = build_vocab(train_df[TEXT_COL])

    @pandas_udf(StringType())
    def predict_udf(texts: pd.Series) -> pd.Series:
        nonlocal model, encoder, vocab
        if model is None:
            import torch
            import joblib

            encoder = joblib.load(encoder_path)
            model = CNNTextClassifier(vocab_size=len(vocab), embed_dim=EMBEDDING_DIM, seq_len=seq_len, num_classes=len(encoder.classes_))
            state_dict = torch.load(model_path, map_location="cpu")
            model.load_state_dict(state_dict)
            model.eval()

        texts_proc = [preprocess_text(t) for t in texts]
        X_seq = texts_to_sequences(texts_proc, vocab, seq_len)  # torch.tensor
        with torch.no_grad():
            logits = model(X_seq)
            preds = torch.argmax(logits, dim=1).numpy()
        preds_original = encoder.inverse_transform(preds)
        return pd.Series([SENT_ID2NAME[int(x)] for x in preds_original])

    return predict_udf

In [35]:
# Load some Spark dataframe (example: test data)
spark_df = spark.read.csv(TEST_PATH, header=True)
df = spark_df.limit(10)

# Add predictions for each aspect
for aspect in ASPECTS:
    predict_udf = make_predict_udf(aspect)
    df = df.withColumn(f"{aspect}_pred", predict_udf(df["Review"]))  # check column name

# Show results
cols_to_show = ["Review"] + [f"{aspect}_pred" for aspect in ASPECTS]
df.select(*cols_to_show).limit(10).show()

+--------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|              Review|Price_pred|Shipping_pred|Outlook_pred|Quality_pred|Size_pred|Shop_Service_pred|General_pred|Others_pred|
+--------------------+----------+-------------+------------+------------+---------+-----------------+------------+-----------+
|Giày hơi có mùi n...|      None|         None|        None|    Negative|     None|             None|        None|       None|
|Hàng về đẹp lắm n...|      None|     Positive|    Positive|        None| Positive|             None|        None|       None|
|Hàng ôk nên mua D...|      None|         None|    Positive|        None|     None|             None|    Positive|       None|
|Bun. GTI gửi Oke ...|      None|         None|        None|        None|     None|             None|        None|       None|
|Màu đẹp giống tro...|      None|         None|    Positive|        None|     None|             None|        No